In [2]:
pip install pulser

Defaulting to user installation because normal site-packages is not writeable
  Using cached pulser-0.18.0-py3-none-any.whl (6.8 kB)
  Using cached pulser_pasqal-0.18.0-py3-none-any.whl (12 kB)
  Using cached pulser_simulation-0.18.0-py3-none-any.whl (34 kB)
  Using cached pulser_core-0.18.0-py3-none-any.whl (179 kB)
  Using cached referencing-0.35.1-py3-none-any.whl (26 kB)
     |████████████████████████████████| 30.3 MB 24.7 MB/s eta 0:00:01
     |████████████████████████████████| 7.3 MB 25.0 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 35.1 MB/s eta 0:00:01
     |████████████████████████████████| 88 kB 18.2 MB/s eta 0:00:01
  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached pasqal_cloud-0.8.1-py3-none-any.whl (49 kB)
  Using cached qutip-4.7.6.tar.gz (3.6 MB)
  distutils: /private/var/folders/jh/s56nmhqx1qld6x7k_sn6jtnc0000gp/T/pip-build-env-svnj0yzu/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /